In [1]:
import numpy as np 
import pandas as pd
from lightgbm import LGBMClassifier
from preprocessing import preprocess

## Preprocess Data

In [ ]:
df = pd.read_csv('data/accepted_2007_to_2018Q4.csv')
df = preprocess(df)